In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
# 모델의 reproducibility를 위해 학습 환경 및 random seed 고정
import numpy as np
import tensorflow as tf
import random

seed_num = 256
np.random.seed(seed_num)
random.seed(seed_num)
tf.random.set_seed(seed_num)

from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1,
                                        inter_op_parallelism_threads = 1)
sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
K.set_session(sess)

In [ ]:
#모델 라이브러리 가져오기 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x_train = np.load('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/melspectogram.npy')

In [ ]:
target = []
for i in range(200):
  target.append(0)
for i in range(200):
  target.append(1)
for i in range(200):
  target.append(2)
for i in range(200):
  target.append(3)
for i in range(200):
  target.append(4)

In [ ]:
y_train = np.array(target)

In [ ]:
import cv2
new = []
for i in range(1000):
  new.append(cv2.resize(x_train[i], (256, 256), interpolation = cv2.INTER_LANCZOS4))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, random_state = 256, test_size = 0.2)

In [ ]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.utils.np_utils import to_categorical

# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3


# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

# y_train = y_data[:1395]
# y_test = y_data[1395:]

# 데이터의 모양 출력하기
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 128, 2584)
(200, 128, 2584)
(800,)
(200,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

# 기본 RNN 모델을 구현하기 위한 함수
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (128, 2584), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/10
16/16 [==============================] - 4s 185ms/step - loss: 1.6947 - accuracy: 0.1787
Epoch 2/10
16/16 [==============================] - 4s 223ms/step - loss: 1.6293 - accuracy: 0.1762
Epoch 3/10
16/16 [==============================] - 4s 267ms/step - loss: 1.6162 - accuracy: 0.2000
Epoch 4/10
16/16 [==============================] - 4s 253ms/step - loss: 1.6139 - accuracy: 0.1800
Epoch 5/10
16/16 [==============================] - 3s 194ms/step - loss: 1.6158 - accuracy: 0.1937
Epoch 6/10
16/16 [==============================] - 3s 194ms/step - loss: 1.6225 - accuracy: 0.1762
Epoch 7/10
16/16 [==============================] - 3s 185ms/step - loss: 1.6198 - accuracy: 0.1937
Epoch 8/10
16/16 [==============================] - 4s 235ms/step - loss: 1.6185 - accuracy: 0.2075
Epoch 9/10
16/16 [==============================] - 3s 184ms/step - loss: 1.6233 - accuracy: 0.1825
Epoch 10/10
16/16 [==============================] - 3s 192ms/step - loss: 1.6239 - accuracy: 0.1688

In [ ]:
# 단층 LSTM을 구현하기 위한 함수
from keras.layers import LSTM

def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (128, 2584), return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/10
16/16 [==============================] - 4s 51ms/step - loss: 1.6532 - accuracy: 0.1887
Epoch 2/10
16/16 [==============================] - 1s 43ms/step - loss: 1.6201 - accuracy: 0.1925
Epoch 3/10
16/16 [==============================] - 1s 41ms/step - loss: 1.6118 - accuracy: 0.1800
Epoch 4/10
16/16 [==============================] - 1s 41ms/step - loss: 1.6097 - accuracy: 0.1725
Epoch 5/10
16/16 [==============================] - 1s 41ms/step - loss: 1.6101 - accuracy: 0.2050
Epoch 6/10
16/16 [==============================] - 1s 41ms/step - loss: 1.6104 - accuracy: 0.2050
Epoch 7/10
16/16 [==============================] - 1s 40ms/step - loss: 1.6102 - accuracy: 0.1975
Epoch 8/10
16/16 [==============================] - 1s 40ms/step - loss: 1.6103 - accuracy: 0.1975
Epoch 9/10
16/16 [==============================] - 1s 41ms/step - loss: 1.6101 - accuracy: 0.2025
Epoch 10/10
16/16 [==============================] - 1s 39ms/step - loss: 1.6103 - accuracy: 0.2037
0.18


In [ ]:
# 다층 LSTM을 구현하기 위한 함수
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (128, 2584), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/10
16/16 [==============================] - 3s 53ms/step - loss: 1.6260 - accuracy: 0.1925
Epoch 2/10
16/16 [==============================] - 1s 46ms/step - loss: 1.6201 - accuracy: 0.1825
Epoch 3/10
16/16 [==============================] - 1s 59ms/step - loss: 1.6124 - accuracy: 0.1988
Epoch 4/10
16/16 [==============================] - 1s 57ms/step - loss: 1.6120 - accuracy: 0.1838
Epoch 5/10
16/16 [==============================] - 1s 61ms/step - loss: 1.6125 - accuracy: 0.2050
Epoch 6/10
16/16 [==============================] - 1s 55ms/step - loss: 1.6152 - accuracy: 0.1475
Epoch 7/10
16/16 [==============================] - 1s 51ms/step - loss: 1.6151 - accuracy: 0.2000
Epoch 8/10
16/16 [==============================] - 1s 52ms/step - loss: 1.6161 - accuracy: 0.2000
Epoch 9/10
16/16 [==============================] - 1s 57ms/step - loss: 1.6138 - accuracy: 0.1825
Epoch 10/10
16/16 [==============================] - 1s 62ms/step - loss: 1.6121 - accuracy: 0.1875
0.18


In [ ]:
# 단층 GRU를 구현하기 위한 함수
from keras.layers import GRU

def gru():
    model = Sequential()
    model.add(GRU(50, input_shape = (128, 2584), return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = gru, epochs = 10, batch_size = 10, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/10
80/80 [==============================] - 2s 15ms/step - loss: 1.8696 - accuracy: 0.1963
Epoch 2/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6193 - accuracy: 0.1750
Epoch 3/10
80/80 [==============================] - 1s 13ms/step - loss: 1.6155 - accuracy: 0.1850
Epoch 4/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6142 - accuracy: 0.1750
Epoch 5/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6119 - accuracy: 0.2075
Epoch 6/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6192 - accuracy: 0.1787
Epoch 7/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6142 - accuracy: 0.2013
Epoch 8/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6136 - accuracy: 0.1925
Epoch 9/10
80/80 [==============================] - 1s 14ms/step - loss: 1.6159 - accuracy: 0.1912
Epoch 10/10
80/80 [==============================] - 1s 13ms/step - loss: 1.6166 - accuracy: 0.1688
0.23
